In [80]:
import gzip
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

from zipfile import ZipFile

In [81]:
with ZipFile("MNIST.zip", 'r') as mnist:
    mnist.extractall("")
    list_files = mnist.namelist()
    print(list_files)

['t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz']


In [82]:
def MNIST():
    with gzip.open("train-images-idx3-ubyte.gz", 'rb') as data:
        _ = int.from_bytes(data.read(4), 'big')
        num_images = int.from_bytes(data.read(4), 'big')
        rows = int.from_bytes(data.read(4), 'big')
        cols = int.from_bytes(data.read(4), 'big')
        train_images = data.read()
        X_train = np.frombuffer(train_images, dtype=np.uint8)
        X_train = X_train.reshape((num_images, rows, cols))
        
    with gzip.open("train-labels-idx1-ubyte.gz", 'rb') as data:
        train_labels = data.read()[8:]
        y_train = np.frombuffer(train_labels, dtype=np.uint8)

    with gzip.open("t10k-images-idx3-ubyte.gz", 'rb') as data:
        _ = int.from_bytes(data.read(4), 'big')
        num_images = int.from_bytes(data.read(4), 'big')
        rows = int.from_bytes(data.read(4), 'big')
        cols = int.from_bytes(data.read(4), 'big')
        test_images = data.read()
        X_test = np.frombuffer(test_images, dtype=np.uint8)
        X_test = X_test.reshape((num_images, rows, cols))

    with gzip.open("t10k-labels-idx1-ubyte.gz", 'rb') as data:
        test_labels = data.read()[8:]
        y_test = np.frombuffer(test_labels, dtype=np.uint8)
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = MNIST()

In [83]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [84]:
X_validate = X_train[50000:].reshape(10000, -1).astype(np.float)/255
y_validate = y_train[50000:].reshape(10000, 1)

X_train = X_train[:50000].reshape(50000, -1).astype(np.float32)/255
y_train = y_train[:50000].reshape(50000, 1)

X_test = X_test.copy().reshape(10000, -1).astype(np.float)/255
y_test = y_test.copy().reshape(10000, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


In [85]:
print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"X validate shape: {X_validate.shape}")
print(f"y validate shape: {y_validate.shape}")
print(f"X test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

X train shape: (50000, 784)
y train shape: (50000, 1)
X validate shape: (10000, 784)
y validate shape: (10000, 1)
X test shape: (10000, 784)
y test shape: (10000, 1)


In [86]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Devide: {device}")

Devide: cuda


In [87]:
class model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = torch.nn.Sequential(
        torch.nn.Linear(784, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 10)
    )
    self.loss = torch.nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.parameters())
    self.to(device)

  def forward(self, X):
    return self.layers(X)

  def fit(self, X, Y):
    self.optimizer.zero_grad()
    y_pred = self.forward(X)
    loss = self.loss(y_pred, Y)
    loss.backward()
    self.optimizer.step()
    return loss.item()

  def predict(self, X):
    with torch.no_grad():
      return torch.argmax(self.forward(X), dim=1)

#TEST 1

In [9]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Lambda(lambda image: image.view(784))
])

data_train = torchvision.datasets.MNIST(root="./", train=True, transform=transform)
data_test = torchvision.datasets.MNIST(root="./", train=False, transform=transform)

dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=16, shuffle=True)

In [11]:
mnist_model = model()

epochs = 5
for i in range(1, epochs+1):
  total_loss = 0
  for xs, ys in dataloader_train:
    xs, ys = xs.to(device), ys.to(device)
    total_loss += mnist_model.fit(xs, ys)
  total_loss /= len(dataloader_train)
  print(f"Epoch {i}: {total_loss:.4f}")

Epoch 1: 0.1948
Epoch 2: 0.0885
Epoch 3: 0.0620
Epoch 4: 0.0490
Epoch 5: 0.0410


In [13]:
correct = 0
for xs, ys in dataloader_test:
  xs, ys = xs.to(device), ys.to(device)
  y_pred = mnist_model.predict(xs)
  correct += (ys == y_pred).sum()
acc = correct/(len(dataloader_test)*16)
print(f"Accuracy: {acc}")

Accuracy: 0.9763999581336975


##TEST 2

In [88]:
X_train = torch.tensor(X_train.copy())
y_train = (torch.tensor(np.squeeze(y_train.copy()))).to(dtype=torch.long)

X_validate = (torch.tensor(X_validate.copy())).to(dtype=torch.float32)
y_validate = (torch.tensor(np.squeeze(y_validate.copy()))).to(dtype=torch.long)

X_test = (torch.tensor(X_test.copy())).to(dtype=torch.float32)
y_test = (torch.tensor(np.squeeze(y_test.copy()))).to(dtype=torch.long)

In [89]:
Xy_train = torch.utils.data.TensorDataset(X_train, y_train)
Xy_validate = torch.utils.data.TensorDataset(X_validate, y_validate)
Xy_test = torch.utils.data.TensorDataset(X_test, y_test)

dataloader_Xy_train = torch.utils.data.DataLoader(Xy_train, batch_size=16, shuffle=True)
dataloader_Xy_validate = torch.utils.data.DataLoader(Xy_validate, batch_size=16, shuffle=True)
dataloader_Xy_test = torch.utils.data.DataLoader(Xy_test, batch_size=16, shuffle=True)

In [90]:
epochs = 5
for i in range(1, epochs+1):
  total_loss = 0
  for xs, ys in dataloader_Xy_train:
    xs, ys = xs.to(device), ys.to(device)
    total_loss += mnist_model.fit(xs, ys)
  total_loss /= len(dataloader_train)
  print(f"Epoch {i}: {total_loss:.4f}")

Epoch 1: 0.0121
Epoch 2: 0.0127
Epoch 3: 0.0131
Epoch 4: 0.0132
Epoch 5: 0.0116


In [91]:
correct = 0
for xv, yv in dataloader_Xy_validate:
  xv, yv = xv.to(device), yv.to(device)
  y_pred = mnist_model.predict(xv)
  correct += (yv == y_pred).sum()
acc = correct/(len(dataloader_test)*16)
print(f"Accuracy: {acc}")

Accuracy: 0.9875999689102173


In [92]:
for xt, yt in dataloader_Xy_test:
  xt, yt = xt.to(device), yt.to(device)
  print(f"Prediction: {mnist_model.predict(xt)}")
  print(f"True:       {yt}")
  break

Prediction: tensor([8, 1, 7, 3, 9, 0, 9, 4, 9, 7, 7, 6, 6, 3, 4, 4], device='cuda:0')
True:       tensor([8, 1, 7, 3, 9, 0, 9, 4, 9, 7, 7, 6, 6, 3, 4, 4], device='cuda:0')
